<h1><center>Laboratorio 10: Persistencia, APIs y Servidores 💦</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Patricio Ortiz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1:Pablo Esteban Zúñiga Varela
- Nombre de alumno 2:

### **Link de repositorio de GitHub:** `https://github.com/PabloZV/Laboratorio-de-Programacion-Cientifica-para-Ciencia-de-Datos`

# Temas a tratar

- Persistencia de modelos.
- APIs Rest.
- Servidores.
- Bases de Datos basadas en JSON (formato similar a diccionarios).

## Reglas:

- Fecha de entrega: 7/07/2021
- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Utilizar `scikit-learn` para generar un modelo y luego guardarlo a un achivo.
- Entender qué es una API y qué significa que esta cumpla con los principios de REST.
- Utilizar `FastAPI` para crear un servicio que permita usar el modelo.
- Guardar llamadas a la API y obtener sus resultados usando una base de datos basada en documentos.


## Introducción: Crear y Persistir un Modelo

<div align='center'/>
  <img src="https://pbs.twimg.com/media/E1jfjTRWYAAfi2v.jpg" width="500">
</div>

Consumida en la tristeza el despido de Renacín, Smapina ha decaído en su desempeño, lo que se ha traducido en un irregular tratamiento del agua. Esto ha implicado una baja en la calidad del agua, llegando a haber algunos puntos de la comuna en la que el vital elemento no es apto para el consumo humano. Es por esto que la sanitaria pública de la municipalidad de Maipú se ha contactado con ustedes para que le entreguen una urgente solución a este problema (a la vez que dejan a Smapina, al igual que Renacín, sin trabajo 😔). 

El problema que la empresa le ha solicitado resolver es el de elaborar un sistema que les permita saber si el agua es potable o no. Para esto, la sanitaria les ha proveido una base de datos con la lectura de múltiples sensores IOT colocados en diversas cañerías, conductos y estanques. Estos sensores señalan nueve tipos de mediciones químicas y más una etiqueta elaborada en laboratorio que indica si el agua es potable o no el agua.

La idea final es que puedan, en el caso que el agua no sea potable, dar un aviso inmediato para corregir el problema. Tenga en cuenta que parte del equipo docente vive en Maipú y su intoxicación podría implicar graves problemas para el cierre del curso.

Atributos:

1. pH value
2. Hardness
3. Solids (Total dissolved solids - TDS)
4. Chloramines
5. Sulfate
6. Conductivity
7. Organic_carbon
8. Trihalomethanes
9. Turbidity

Variable a predecir: 

10. Potability (1 si es potable, 0 no potable)

\[Fuera de rol\] Los datos originales, junto a la descripción de cada atributo se encuentran en: 

https://www.kaggle.com/adityakadiwal/water-potability

## Modelo a Utilizar

<div align="center">
  <img src="https://upload.wikimedia.org/wikipedia/commons/6/6c/SMAPA.jpg" width="400">
</div>

Al recibir el trabajo, se dan cuenta que el equipo de TI de la sanitaria, de forma autodidacta, ya estaba trabajando en un proyecto de predicción de agua potable utilizando *Machine Learning*. Sí bien el código no es el mejor, ustedes como equipo consideran que el trabajo de predicción ya está realizado y prefieren utilizar todos sus recursos en la ardua tarea de desplegarlo.

Por lo mismo, lea y ejecute el código proveido para entrenar y guardar el modelo y luego continue con el despliegue del sistema (No debe hacer nada en esta parte, solo entender el problema que se quiere resolver).

In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import classification_report
from sklearn.experimental import enable_halving_search_cv 
from sklearn.model_selection import HalvingGridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [3]:
#from google.colab import drive
# ejecutar en el caso de usar colab.
#drive.mount("/content/drive")

In [2]:
df = pd.read_csv("./water_potability.csv").dropna()
df

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
3,8.316766,214.373394,22018.417441,8.059332,356.886136,363.266516,18.436524,100.341674,4.628771,0
4,9.092223,181.101509,17978.986339,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075,0
5,5.584087,188.313324,28748.687739,7.544869,326.678363,280.467916,8.399735,54.917862,2.559708,0
6,10.223862,248.071735,28749.716544,7.513408,393.663396,283.651634,13.789695,84.603556,2.672989,0
7,8.635849,203.361523,13672.091764,4.563009,303.309771,474.607645,12.363817,62.798309,4.401425,0
...,...,...,...,...,...,...,...,...,...,...
3267,8.989900,215.047358,15921.412018,6.297312,312.931022,390.410231,9.899115,55.069304,4.613843,1
3268,6.702547,207.321086,17246.920347,7.708117,304.510230,329.266002,16.217303,28.878601,3.442983,1
3269,11.491011,94.812545,37188.826022,9.263166,258.930600,439.893618,16.172755,41.558501,4.369264,1
3270,6.069616,186.659040,26138.780191,7.747547,345.700257,415.886955,12.067620,60.419921,3.669712,1


In [3]:
X = df.drop(columns=["Potability"])
y = df.loc[:, ["Potability"]].values.ravel()

# Holdout
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42, shuffle=True, stratify=y
)

# Pipeline
pipe = Pipeline(
    [
        ("Scaler", StandardScaler()),
        ("Clf", BaggingClassifier()),
    ]
)

# Grilla de Parámetros
param_grid = {
    "Clf__max_features": [0.5, 0.7, 1.0],
    "Clf__max_samples": [0.5, 0.7, 1.0],
    "Clf__n_estimators": [5, 10, 15, 20, 25, 50],
}

# Buscar la mejor configuración de parámetros usando HalvingGridSearchCV
gs = HalvingGridSearchCV(pipe, param_grid, verbose=100, cv=5, n_jobs=-1)
gs.fit(X_train, y_train)

gs


n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 55
max_resources_: 1508
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 54
n_resources: 55
Fitting 5 folds for each of 54 candidates, totalling 270 fits
[CV 1/5; 1/54] START Clf__max_features=0.5, Clf__max_samples=0.5, Clf__n_estimators=5
[CV 1/5; 1/54] END Clf__max_features=0.5, Clf__max_samples=0.5, Clf__n_estimators=5;, score=(train=0.791, test=0.455) total time=   0.0s
[CV 2/5; 2/54] START Clf__max_features=0.5, Clf__max_samples=0.5, Clf__n_estimators=10
[CV 1/5; 2/54] START Clf__max_features=0.5, Clf__max_samples=0.5, Clf__n_estimators=10
[CV 4/5; 2/54] START Clf__max_features=0.5, Clf__max_samples=0.5, Clf__n_estimators=10
[CV 4/5; 1/54] START Clf__max_features=0.5, Clf__max_samples=0.5, Clf__n_estimators=5
[CV 1/5; 2/54] END Clf__max_features=0.5, Clf__max_samples=0.5, Clf__n_estimators=10;, score=(train=0.791, test=0.364) total time=   0.0s[CV 4/5; 2/54] END Clf__max_featu

HalvingGridSearchCV(estimator=Pipeline(steps=[('Scaler', StandardScaler()),
                                              ('Clf', BaggingClassifier())]),
                    n_jobs=-1,
                    param_grid={'Clf__max_features': [0.5, 0.7, 1.0],
                                'Clf__max_samples': [0.5, 0.7, 1.0],
                                'Clf__n_estimators': [5, 10, 15, 20, 25, 50]},
                    verbose=100)

In [4]:
print(f"Mejor Modelo Grid Search: {gs.best_estimator_}\n")
print(f"Mejores Parámetros: {gs.best_params_}\n")

print(f"Mejor Accuracy Grid Search: {gs.best_score_}\n")

best_random_forest = gs.best_estimator_
y_pred = best_random_forest.predict(X_test)

print("Reporte de Clasificación\n")
print(classification_report(y_test, y_pred))

Mejor Modelo Grid Search: Pipeline(steps=[('Scaler', StandardScaler()),
                ('Clf', BaggingClassifier(max_samples=0.7, n_estimators=50))])

Mejores Parámetros: {'Clf__max_features': 1.0, 'Clf__max_samples': 0.7, 'Clf__n_estimators': 50}

Mejor Accuracy Grid Search: 0.6641755391755391

Reporte de Clasificación

              precision    recall  f1-score   support

           0       0.70      0.89      0.78       300
           1       0.72      0.42      0.53       203

    accuracy                           0.70       503
   macro avg       0.71      0.66      0.66       503
weighted avg       0.71      0.70      0.68       503



Y use `dump` de `joblib` para persistir el modelo recién entrenado en el disco.

In [5]:
from joblib import dump, load

dump(gs, "modelo.joblib")

['modelo.joblib']

## Parte 1: Conceptos \[3 Puntos\]

<div align="center">
  <img src="https://media1.tenor.com/images/14c1bcb6e0eb7c5ff4ac225f1114e819/tenor.gif?itemid=13036231" width="400">
</div>

Antes de comenzar a trabajar, la administración de la empresa de agua les pide explicar la ideas claves para el desarrollo de una aplicación web. 

Para esto, les pide explicar muy brevemente los siguientes conceptos:

**Nota:** Puede ser de ayuda revisar la clase 24. 

> ¿Qué es el modelo Cliente-Servidor?

        Un servidor responde las solicitudes de varios clientes. El cliente normalmente le facilita un input, el servidor usaese input para realizar un a acción que puede o no ser una respuesta al cliente.
    
> ¿Qué es una URI?

        Es un aserie de caracteres que localiza un recurso web
    
    
> ¿Qué es HTTP?¿Qué significa que sea stateless?

        Hypertext Transfer Protocol, es un protocolo de transferencia de datos, es el más comunmente usado en internet.
    
> ¿En que consisten las operaciones de HTTP?: 
   
- GET
    
        Solicita obtener información del servidor
        
- POST
    
        Solicita crear información en el servidor, también se utiliza para solicitar información con autenticación.
        
- PUT
    
        Solicita actualizar información en el servidor
        
- DELETE
    
        Solicita eliminar información en el servidor
        

> ¿Qué indican los códigos de respuesta de HTTP?


        El estado de la solicitud, por ejemplo 200 es OK, 404 es not found, 409 es conflict, etc..

> ¿Qué es un Hipermedio como HTML?

        Es un conjunto de varios medios unidos entre si para transmitir información

> ¿Qué es una API?

        Application Programming Interface es una interfaz entre dos softwares, es una forma de qeu un software use funcionalidades de otro software sin tener que entender cómo está implementado. Nadie programa un procesador directamente con lenguaje de máquina, el software que está entre un sistema operativo y el fierro es un a API. Una librería como keras que implementa varias funcionalidades de otras librerías de manera más fácil de usar es una API de esas librerías. Hay varios otros ejemplos.

> ¿Cómo API REST engloba los términos anteriores?

        Representational State Transfer (REST). API REST es una forma comercial de decir API HTTP para la web.
         
> Indique si sería buena idea o no almacenar las observaciones y predicciones que se efectuen en el servicio de predicción de agua potable y el por qué.

       Sí, para saber si el modelo lo sigue haciendo bien en el futuro o hay que modificarlo.

---
## Parte 2: Implementación del Servidor [3 Puntos]


<div align="center">
  <img src="https://media3.giphy.com/media/YQitE4YNQNahy/giphy-downsized-large.gif" width="500">
</div>

La municipal de agua les pide variadas funcionalidades al servidor que crearán. Su única restricción es que la implementen usando `FastAPI`.

Junto a la tarea se les entrega además un archivo main.py que implementa el servidor en donde están definidas las funciones que deben implementar. Ustedes deben implementar el contenido de estas funciones.

Instrucciones iniciales:

- Para instalar el servidor: `pip install fastapi[all]`. 
- Para iniciarlo, en la consola de su archivo principal, ejecutar `uvicorn main:app --reload`. 
- El servidor se autorecarga cada vez que guardan un cambio en el código. 

> Nota: Recuerden usar las documentaciones para probar las APIs que desarrollen: http://127.0.0.1:8000/docs#/

> Nota 2: Les aconsejamos seriamente intentar usar el debuger de vscode para programar el servidor.

#### 1. \[1 Punto\] Predicción con `POST`

A través de un `POST` a la ruta `/potabilidad` prediga si una medición de agua es o no potable. Una llamada a esta ruta debe además registrar los datos entregados más la fecha de la predicción y la clase predicha en una base de datos (ver la sección de `TinyDB` para mayor información).


Por ejemplo, una llamada de esta ruta con un Body: 


```json
{
   "ph":10.316400384553162,
   "Hardness":217.2668424334475,
   "Solids":10676.508475429378,
   "Chloramines":3.445514571005745,
   "Sulfate":397.7549459751925,
   "Conductivity":492.20647361771086,
   "Organic_carbon":12.812732207582542,
   "Trihalomethanes":72.28192021570328,
   "Turbidity":3.4073494284238364
}```

Su serivdor debería retornar una respuesta HTML con código 200 con: 


```json
{
  "potabilidad": 0,
  "id": 121 # el id/índice en donde este elemento fue insertado en la BBDD.
}
```

Nota: puede variar según el ejemplo y el clasificador que entrenen. 


#### 2. \[0.5 Puntos\] Obtener Datos con `Get`


A través de un `GET` a la ruta `/potabilidad/` entrege todas las mediciones guardadas.


Por ejemplo: 

http://127.0.0.1:8000/potabilidad

Debería retornar todos los registros con su fecha y predicción

```json
[
   {
      "ph":10.316400384553162,
      "Hardness":217.2668424334475,
      "Solids":10676.508475429378,
      "Chloramines":3.445514571005745,
      "Sulfate":397.7549459751925,
      "Conductivity":492.20647361771086,
      "Organic_carbon":12.812732207582542,
      "Trihalomethanes":72.28192021570328,
      "Turbidity":3.4073494284238364,
      "Day":17,
      "Month":7,
      "Year":2021,
      "Prediction":0
   },
   {
      "ph":6.599593818953803,
      "Hardness":198.323260816452,
      "Solids":21078.887166441476,
      "Chloramines":6.23423148205243,
      "Sulfate":331.49563679988063,
      "Conductivity":447.9877914293426,
      "Organic_carbon":12.93630432875392,
      "Trihalomethanes":59.833619304688185,
      "Turbidity":4.702310752770497,
      "Day":17,
      "Month":7,
      "Year":2021,
      "Prediction":0
   },
   {
      "ph":7.920794564373236,
      "Hardness":173.35715056714022,
      "Solids":13454.634777245552,
      "Chloramines":6.947895137367302,
      "Sulfate":356.768119344542,
      "Conductivity":471.3686945543212,
      "Organic_carbon":16.47783972341182,
      "Trihalomethanes":68.00050835002364,
      "Turbidity":3.3743265226271983,
      "Day":17,
      "Month":7,
      "Year":2021,
      "Prediction":0
   }, 
   ...
]
```

Nota: Esta es bastante sencilla, no debería tomarles más de 2 lineas.

#### 3. \[0.5 Puntos\] `GET` con parámetros
 A través de un `GET` a la ruta `/potabilidad_diaria/` con los parámetros día, mes y año entrege todas las mediciones guardadas correspondientes a esa fecha.

Por ejemplo: 

http://127.0.0.1:8000/potabilidad/?day=17&month=7&year=2021

Debería entregar las mediciones asignadas al día 17 del mes 7 del año 2021:

```json

[
    {"ph":10.316400384553162,"Hardness":217.2668424334475,"Solids":10676.508475429378,"Chloramines":3.445514571005745,"Sulfate":397.7549459751925,"Conductivity":492.20647361771086,"Organic_carbon":12.812732207582542,"Trihalomethanes":72.28192021570328,"Turbidity":3.4073494284238364,"Day":17,"Month":7,"Year":2021,"Prediction":0},{"ph":6.599593818953803,"Hardness":198.323260816452,"Solids":21078.887166441476,"Chloramines":6.23423148205243,"Sulfate":331.49563679988063,"Conductivity":447.9877914293426,"Organic_carbon":12.93630432875392,"Trihalomethanes":59.833619304688185,"Turbidity":4.702310752770497,"Day":17,"Month":7,"Year":2021,"Prediction":0},{"ph":7.920794564373236,"Hardness":173.35715056714022,"Solids":13454.634777245552,"Chloramines":6.947895137367302,"Sulfate":356.768119344542,"Conductivity":471.3686945543212,"Organic_carbon":16.47783972341182,"Trihalomethanes":68.00050835002364,"Turbidity":3.3743265226271983,"Day":17,"Month":7,"Year":2021,"Prediction":0},
...
]
```

**NOTA**: En la BBDD de prueba existen registros para los días 14, 15, 16, 17, 18 y 19 del 7 de 2021.



#### 4. \[0.5 Puntos\] `PUT` con parámetros:

A través de un `PUT` a la ruta `/potabilidad/`, con los parámetros día, mes, año y clase nueva, actualize la potabilidad de las observaciones según lo entregado.

Ejemplo: 

http://127.0.0.1:8000/potabilidad/?day=17&month=7&year=2021&new_prediction=1

Debería actualizar la predicción de todos los registros del día 17/07/2021 a la clase 1 y retornar si fue exitoso o no y una lista de los documentos actualizados.


```json
{
  "success": true,
  "updated_elements": [
    61,
    62,
    63,
    64,
    65,
    66,
    67,
    68,
    69,
    70,
    71,
    72,
    73,
    74,
    75,
    76,
    77,
    78,
    79,
    80
  ]
}
```

#### 5. \[0.5 Puntos\] `DELETE` con parámetros:

A través de un `DELETE` a la ruta `/potabilidad/`, con parámetros día, mes y año, elimine todas las observaciones de ese día.

Ejemplo: 
    
http://127.0.0.1:8000/potabilidad/?day=17&month=7&year=2021
    
Debería retornar: 
    
    
```json
{
  "success": true,
  "deleted_elements": [
    61,
    62,
    63,
    64,
    65,
    66,
    67,
    68,
    69,
    70,
    71,
    72,
    73,
    74,
    75,
    76,
    77,
    78,
    79,
    80
  ]
}

```

----

### Paréntesis: Tutorial BBDD y `TinyDB`


Un sistema gestor de base de datos (SGBD) según wikipedia es: 
 
> Un conjunto de programas que permiten el almacenamiento, modificación y extracción de la información en una base de datos.

En general, es la interfaz con la cual nos comunicamos con la BBDD y que a la vez, administra los datos y todo lo relacionado con estos.

---

`TinyDB` es un gestor de base de datos minimalista para Python orientada a documentos. Es decir, está enfocada a guardar datos en forma de diccionarios. En nuestro caso, la usaremos como ejemplo para el lab, pero en el caso general, es recomendable usar mejores gestores de BBDD de documentos como MongoDB (la cual, se parece mucho a esta).


A continuación un ejemplo de uso (que pueden usar junto al desarrollo de la tarea)

In [7]:
!pip install tinydb

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [8]:
from datetime import datetime

# obtenemos un ejemplo de
ejemplo = df.sample()

# agregamos la fecha
hoy = datetime.now()
ejemplo["Day"] = hoy.day
ejemplo["Month"] = hoy.month
ejemplo["Year"] = hoy.year

In [9]:
ejemplo_dict = ejemplo.to_dict(orient="records")  # convertimos a diccionario
ejemplo_dict  # tinidb puede trabajar con este formato de datos

[{'ph': 5.1959206693648685,
  'Hardness': 175.1571552273615,
  'Solids': 33473.11251835836,
  'Chloramines': 8.407449953610357,
  'Sulfate': 319.253631429962,
  'Conductivity': 484.27989018962126,
  'Organic_carbon': 16.812766967214372,
  'Trihalomethanes': 94.35695426785966,
  'Turbidity': 5.375901893717908,
  'Potability': 0,
  'Day': 3,
  'Month': 12,
  'Year': 2021}]

Primero, creamos un cliente que cargue la base de datos (que ya contiene datos del agua).


In [10]:
from tinydb import Query, TinyDB

db = TinyDB("./db_ejemplo.json")

Para insertar el ejemplo en la base de datos, usamos `insert`

In [11]:
db.insert(
    ejemplo_dict[0]
)  # insertamos solo el diccionario, no la lista con el diccionario.

# el número que imprime (121) indica id (indice) interno del documento recién insertado.

122

Para pedir todos los datos almacenados, usamos `all()`

In [12]:
db.all()[0:10] # limitamos de 0 10 para que solo se muestren 10 ejemplos

[{'ph': 9.485832873402854,
  'Hardness': 218.7382355335101,
  'Solids': 15357.83337292032,
  'Chloramines': 8.534458442405425,
  'Sulfate': 315.83139625832393,
  'Conductivity': 429.6222121825796,
  'Organic_carbon': 15.621138695886827,
  'Trihalomethanes': 64.8260282011821,
  'Turbidity': 3.221129818784097,
  'Day': 14,
  'Month': 7,
  'Year': 2021,
  'Prediction': 1},
 {'ph': 6.435722716949114,
  'Hardness': 214.72466925844697,
  'Solids': 9363.85398761785,
  'Chloramines': 9.99762356652512,
  'Sulfate': 273.4631393655443,
  'Conductivity': 508.34520701170817,
  'Organic_carbon': 12.925869662663729,
  'Trihalomethanes': 71.42858985087607,
  'Turbidity': 4.925339107073759,
  'Day': 14,
  'Month': 7,
  'Year': 2021,
  'Prediction': 1},
 {'ph': 6.365497241052185,
  'Hardness': 205.0565357935376,
  'Solids': 7040.737620777599,
  'Chloramines': 11.30283118849055,
  'Sulfate': 325.32854340517343,
  'Conductivity': 371.5796340409174,
  'Organic_carbon': 16.352858005837504,
  'Trihalomethane

#### Consultas

Podemos hacer consultas, es decir, obtener algunos archivos según algún criterio, a través un objeto `Query` y `search`:

In [13]:
Mediciones = Query()

# Aquí Mediciones.Day representa a todas las llaves Day de todos los documentos.
# luego, el criterio de comparación es el que indica que datos se traeran.
# en la práctica, trae 40 datos, del día 14 al 15.
db.search(Mediciones.Day <= 15)

[{'ph': 9.485832873402854,
  'Hardness': 218.7382355335101,
  'Solids': 15357.83337292032,
  'Chloramines': 8.534458442405425,
  'Sulfate': 315.83139625832393,
  'Conductivity': 429.6222121825796,
  'Organic_carbon': 15.621138695886827,
  'Trihalomethanes': 64.8260282011821,
  'Turbidity': 3.221129818784097,
  'Day': 14,
  'Month': 7,
  'Year': 2021,
  'Prediction': 1},
 {'ph': 6.435722716949114,
  'Hardness': 214.72466925844697,
  'Solids': 9363.85398761785,
  'Chloramines': 9.99762356652512,
  'Sulfate': 273.4631393655443,
  'Conductivity': 508.34520701170817,
  'Organic_carbon': 12.925869662663729,
  'Trihalomethanes': 71.42858985087607,
  'Turbidity': 4.925339107073759,
  'Day': 14,
  'Month': 7,
  'Year': 2021,
  'Prediction': 1},
 {'ph': 6.365497241052185,
  'Hardness': 205.0565357935376,
  'Solids': 7040.737620777599,
  'Chloramines': 11.30283118849055,
  'Sulfate': 325.32854340517343,
  'Conductivity': 371.5796340409174,
  'Organic_carbon': 16.352858005837504,
  'Trihalomethane

Además, podemos mezclar distintos criterios de forma muy similar como lo hacemos en pandas:

En este caso, la siguiente consulta entrega todas las mediciones con ph entre 6 y 6.5 incluidas.

In [14]:
Mediciones = Query()

db.search((Mediciones.ph >= 6) & (Mediciones.ph <= 6.5))

[{'ph': 6.435722716949114,
  'Hardness': 214.72466925844697,
  'Solids': 9363.85398761785,
  'Chloramines': 9.99762356652512,
  'Sulfate': 273.4631393655443,
  'Conductivity': 508.34520701170817,
  'Organic_carbon': 12.925869662663729,
  'Trihalomethanes': 71.42858985087607,
  'Turbidity': 4.925339107073759,
  'Day': 14,
  'Month': 7,
  'Year': 2021,
  'Prediction': 1},
 {'ph': 6.365497241052185,
  'Hardness': 205.0565357935376,
  'Solids': 7040.737620777599,
  'Chloramines': 11.30283118849055,
  'Sulfate': 325.32854340517343,
  'Conductivity': 371.5796340409174,
  'Organic_carbon': 16.352858005837504,
  'Trihalomethanes': 74.31259211793054,
  'Turbidity': 5.125416576401367,
  'Day': 14,
  'Month': 7,
  'Year': 2021,
  'Prediction': 1},
 {'ph': 6.400256164229511,
  'Hardness': 172.2990001423738,
  'Solids': 15936.247359864004,
  'Chloramines': 4.935997766107353,
  'Sulfate': 340.3929303663409,
  'Conductivity': 435.4056098362427,
  'Organic_carbon': 11.075621804849504,
  'Trihalomethan

### Update

Podemos actualizar datos que hayan sido seleccionados usando una opración y una `Query` como argumentos de `update`.

Las operaciones disponibles son:

    delete(key): delete a key from the document
    increment(key): increment the value of a key
    decrement(key): decrement the value of a key
    add(key, value): add value to the value of a key (also works for strings)
    subtract(key, value): subtract value from the value of a key
    set(key, value): set key to value


In [15]:
from tinydb.operations import set

Mediciones = Query()

# en este caso, usaremos las mediciones del día 14 y fijaremos su ph a 7
db.update(set("ph", 7), Mediciones.Day == 14)
# noten que esta función retorna los indices de los documentos que fueron actualizados.

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [16]:
db.search(Mediciones.Day == 14)

[{'ph': 7,
  'Hardness': 218.7382355335101,
  'Solids': 15357.83337292032,
  'Chloramines': 8.534458442405425,
  'Sulfate': 315.83139625832393,
  'Conductivity': 429.6222121825796,
  'Organic_carbon': 15.621138695886827,
  'Trihalomethanes': 64.8260282011821,
  'Turbidity': 3.221129818784097,
  'Day': 14,
  'Month': 7,
  'Year': 2021,
  'Prediction': 1},
 {'ph': 7,
  'Hardness': 214.72466925844697,
  'Solids': 9363.85398761785,
  'Chloramines': 9.99762356652512,
  'Sulfate': 273.4631393655443,
  'Conductivity': 508.34520701170817,
  'Organic_carbon': 12.925869662663729,
  'Trihalomethanes': 71.42858985087607,
  'Turbidity': 4.925339107073759,
  'Day': 14,
  'Month': 7,
  'Year': 2021,
  'Prediction': 1},
 {'ph': 7,
  'Hardness': 205.0565357935376,
  'Solids': 7040.737620777599,
  'Chloramines': 11.30283118849055,
  'Sulfate': 325.32854340517343,
  'Conductivity': 371.5796340409174,
  'Organic_carbon': 16.352858005837504,
  'Trihalomethanes': 74.31259211793054,
  'Turbidity': 5.12541657

### Eliminado


Por último, podemos eliminar de la BBDD con una `Query` y la función `remove`: 

In [17]:
db.remove(Mediciones.Day == 14)
# noten que esta función retorna los indices de los documentos que fueron borrados.

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [18]:
db.search(Mediciones.Day == 14)

[]

### NOTA: Mutación de la BBDD y Respaldo

Cada vez que ejecutan insert, update o remove modifican las bases de datos. El archivo `db_resplado.json` contiene un respaldo de la base de datos para que puedan volver a utilizar la original.

-----

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<div align="center">
  <img src="https://i.pinimg.com/originals/84/5d/f1/845df1aefc6a5e37ae575327a0cc6e43.gif" width="500">
</div>